# Ejercicio con kNNs - Predicción de diabetes
Primero, importamos todos los módulos necesarios

In [ ]:
# Nuestros sospechosos habituales
import pandas as pd
import numpy as np

# También, como siempre, nos apoyamos en Scikit-Learn para hacer el split en training y test
from sklearn.model_selection import train_test_split
# Usaremos el preprocesador StandardScaler para no tener sesgos en los datos
# de entrada
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
# Importamos herramientas para evaluar el modelo. F1 es la media
# armónica de precision y recall
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

Descargamos un dataset con información de Diabetes para nuestro ejercicio:

In [ ]:
!curl -o data/diabetes.csv https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv

Ahora es el turno de cargar el dataset en un Pandas dataframe y de investigar un poco qué es lo que tenemos en términos de número de muestras y de features:

In [ ]:
# Carga del dataset desde un fichero CSV
dataset = pd.read_csv('../data/diabetes.csv')
# Veamos cuántas muestras tiene nuestro dataset
print(len(dataset))
# Miremos también qué pinta tiene el dataset desde el
# punto de vista de características y etiquetas
dataset.head()

Nuestro dataset contiene valores que son nulos, y estos no representan realmente una medida, sino una falta de la misma. Dicho de otra forma, se ha colocado un cero en columnas como grosor de la piel o cantidad de glucosa para marcar que no tenemos ese dato, pese a que el valor cero para estas cantidades es imposible.

De manera exploratoria, vamos a ver para el caso de la característica *Glucose*, las columnas que tienen valor 0:

In [ ]:
dataset.loc[dataset['Glucose'] == 0]

Podemos saber qué columnas tienen valores cero en nuestro dataframe filtrándolas y luego sumando el resultado:

In [ ]:
dataset.isin([0]).sum()

Podríamos prescindir de esas muestras para el entrenamiento de nuestro modelo, pero otra técnica que podemos utilizar y que no implica reducir el tamaño del dataset, es sustituir esos valores por el valor medio de la característica en cuestión en todo el dataset.

En el caso de la glucosa, veamos cuál es el valor medio calculado a partir de las muestras en el dataset:

In [ ]:
dataset['Glucose'].mean

In [ ]:
# Sustituimos los ceros para características en las que en realidad
# el valor 0 significa que en realidad no tenemos datos
# Primero listamos las columnas en las que queremos aplicar la transformación
zero_not_accepted=['Glucose','BloodPressure','SkinThickness','BMI','Insulin']
# Ahora iteramos para cada una de estas columnas, para poner la media
# 
for column in zero_not_accepted:
    # Usamos Numpy NaN para marcar que el dato no existe, "no data"
    # Según la documentación oficial de Numpy:
    # "NaNs can be used as a poor-man’s mask (if you don’t care what the original value was)""
    dataset[column] = dataset[column].replace(0,np.NaN)
    # Ahora, calculamos la media de cada columna seleccionada, 
    # ignorando la máscara NaN con la opción de Pandas skipna
    mean = int(dataset[column].mean(skipna=True))

    dataset[column]=dataset[column].replace(np.NaN,mean)

In [ ]:
# Echamos un vistazo al dataset y vemos que ahora no tenemos ningún valor distinto de cero
print(dataset['Glucose'])

Le pedimos a Pandas de manera temporal que muestre el dataset completo al pedírselo, sin límite en las columnas. De esta manera podemos comprobar de un vistazo que nuestras transformaciones preparatorias de los datos se han completado con éxito:

In [ ]:
pd.set_option('display.max_rows', None)
display(dataset)

Volvemos a configurar las opciones de visualización de los Pandas dataframes a su valor por defecto, para no ocupar demasiado espacio en el cuaderno Jupyter en nuestras siguientes operaciones exploratorias sobre el dataset:

In [ ]:
pd.reset_option('display.max_rows')

Antes de comenzar con el proceso de entrenamiento, como siempre, partimos el dataset en training y test. Esta vez elegimos que un 20% de nuestras muestras vayan destinadas a nuestra dataset de tests: 

In [ ]:
# Miramos el i location del pandas dataset en todas las filas, y cogemos
# de la columna 0 a la 8. Recordemos que la columna nueve (índice 8) es la que tiene
# nuestras etiquetas, y también que Python cuenta desde cero y que los slices de listas
# no incluyen la última columna.
X = dataset.iloc[:,0:8]
y = dataset.iloc[:,8]
# Seleccionamos un 20% del dataset original como datos de test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.2)

Con la preparación de los datos previa no es suficiente para poder comenzar el entrenamiento. Ahora nos enfrentamos al problema de tener variables que no expresan de manera correcta relaciones que puedan ser entendidas por kNN. 

Consider a simple two class classification problem, where a Class 1 sample is chosen (black) along with it's 10-nearest neighbors (filled green). In the first figure, data is not normalized, whereas in the second one it is.

kNN se apoya como ya hemos visto en un algoritmo de selección de la clase correcta basada en la pertenencia de clase de los k vecinos más próximos para el punto en el que estamos haciendo la predicción, con la medida de proximidad utilizando habitualmente la distancia euclidea (Minkowski con $q=2$).

Si los datos de las diferentes variables numéricas no están normalizados, introduciremos efectos que nos llevan a no realizar la predicción correctamente. Para ilustrar el problema, véase lo que ocurre cuando las escalas de dos variables en un dataset sencillo no están normalizadas:

![Sin normalización](images/No_Normalization.png)

Como puede verse, los datos están comprimidos en una línea en el eje de abcisas debido a la diferente escala del rango de datos (todos se encuentran en el intervalo $(-2,5)$ aproximadamente). Sin embargo, la amplitud de intervalo del eje de ordenadas es mayor, y esto es lo que provoca la agrupación de datos en una forma de línea, llevando a una distorsión de la medida de la distancia que nos llevará a predicciones erróneas.

Tras realizar la normalización mediante la funcion `StandardScaler` de Scikit-Learn, vemos que la distribución de datos cambia, y nos lleva a una predicción distinta de nuestra muestra de ejemplo:

![Con normalización](images/Normalization.png)

Así, pues, apliquemos dicha transformación sobre nuestros datasets antes de proceder al entrenamiento:

In [ ]:
# Ahora escalamos los datos, de manera que todos los rangos van desde -1 hasta 1.
sc_X = StandardScaler()
# Hacemos training y transformación conjunta sobre el training set
X_train = sc_X.fit_transform(X_train)
# Tenemos que asegurarnos de que el testing set también está transformado
X_test = sc_X.transform(X_test)

In [ ]:
# Estimamos el número de vecinos que tenemos que utilizar
import math 
math.sqrt(len(y_test))

In [ ]:
# Definimos el modelo, inicializando kNN con los datos seleccionados
cls = KNeighborsClassifier(n_neighbors=11, p=2, metric='euclidean')
# Entrenamos el modelo
cls.fit(X_train,y_train)

Finalmente, evaluamos el modelo:    

In [ ]:
# Predict the test set results
y_pred = cls.predict(X_test)
y_pred

Para la matriz de confusión, valores reales va en las filas, valores predichos en las columnas. Lo importante es la diagonal, 

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

De nuevo, `f1_score` se define como la media armónica de precision y recall, esto es, $\mathrm{F_1} = 2\frac{\mathrm{precision}\cdot\mathrm{recall}}{\mathrm{precision}+\mathrm{recall}}$

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
print(f1_score(y_test, y_pred))

Ya que estamos trabajando con un modelo médico, lo mejor que podemos hacer es mostrar también el recall o sensitivity del modelo:

In [ ]:
print(recall_score(y_test, y_pred))